#**Sentiment Analysis on Movie Reviews 電影評論情緒分類**
*   0 → negative 負面
*   1 → Somewhat negative  一點點負面
*   2 → Neutral 一般
*   3 → Somewhat positive 一點點正面
*   4 → Positive 正面










### pytorch環境設定(https://pytorch.org)
選擇適合的版本並複製指令貼到本專案

In [21]:
!pip install torch torchvision

In [22]:
#確定有安裝成功
import torch
print(torch.__version__)
torch.cuda.is_available()

1.4.0


True

### **取得 google drive 存取權限**

In [23]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [24]:
cd /content/drive/My Drive/Colab Notebooks/sentiment-analysis-on-movie-reviews/

/content/drive/My Drive/Colab Notebooks/sentiment-analysis-on-movie-reviews


In [25]:
!ls

data   predict.ipynb  train_data.txt  train.ipynb
model  README.md      trained_model   train.tsv


In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### **下載dataset**

In [27]:
import os
def fileExists(filepath):
  return  os.path.exists(filepath)
#我把資料集放到我的github
if fileExists('test.tsv')!=True:
  !wget https://raw.githubusercontent.com/harry83528/sentiment-analysis-on-movie-reviews/master/data/test.tsv
if fileExists('sampleSubmission.csv')!=True:
  !wget https://raw.githubusercontent.com/harry83528/sentiment-analysis-on-movie-reviews/master/data/sampleSubmission.csv

--2020-04-21 14:18:54--  https://raw.githubusercontent.com/harry83528/sentiment-analysis-on-movie-reviews/master/data/test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3367149 (3.2M) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>]   3.21M  --.-KB/s    in 0.1s    

2020-04-21 14:18:54 (32.4 MB/s) - ‘test.tsv’ saved [3367149/3367149]

--2020-04-21 14:18:56--  https://raw.githubusercontent.com/harry83528/sentiment-analysis-on-movie-reviews/master/data/sampleSubmission.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response.

In [28]:
!ls

data   predict.ipynb  sampleSubmission.csv  train_data.txt  train.ipynb
model  README.md      test.tsv		    trained_model   train.tsv


In [29]:
import os
import pandas as pd
df_train = pd.read_csv("test.tsv", sep='\t')
df_train.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [30]:
df_train.shape

(66292, 3)

In [31]:
df_train.columns

Index(['PhraseId', 'SentenceId', 'Phrase'], dtype='object')

In [32]:
df_train.index

RangeIndex(start=0, stop=66292, step=1)

In [33]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66292 entries, 0 to 66291
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PhraseId    66292 non-null  int64 
 1   SentenceId  66292 non-null  int64 
 2   Phrase      66292 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [34]:
df_train.describe()

,PhraseId,SentenceId
count,66292.00000,66292.000000
mean,189206.50000,10114.909144
std,19136.99636,966.787807
min,156061.00000,8545.000000
25%,172633.75000,9266.000000
50%,189206.50000,10086.000000
75%,205779.25000,10941.000000
max,222352.00000,11855.000000


### **將預測資料tsv檔轉成txt檔(【\t】分隔欄位)**



In [0]:
# 將要訓練的句子存起來
def SaveSentence(filepath, sent_list):
    f = open(filepath, 'w', encoding='UTF-8')
    for sent in sent_list:
        f.write(sent + '\n')
    f.close()

# 分割train_data和test_data
def data_Split(FileName):

    fp = open(FileName, 'r', encoding='utf-8')
    line = fp.readline()        # 第一行是表頭PhraseId,SentenceId,Phrase,Sentiment
    line = fp.readline()        # 實際資料的第一行

    test_data = []

    # 用 while 逐行讀取檔案內容(一次讀一行)，直至檔案結尾
    while line:
        sent = ''
        sent = line.replace('\n', '')
        test_data.append(sent)
        line = fp.readline()  #位移到下一行
    
    fp.close()
    
    SaveSentence('predict_data.txt', test_data)

In [0]:
if fileExists('predict_data.txt')!=True:
  data_Split('test.tsv')

### **安裝所需的函式庫-HuggingFace 團隊將 GitHub 專案大翻新並更名成 transformers(Install the Hugging Face Library)**
https://github.com/huggingface/transformers

In [37]:
!pip install transformers

In [0]:
%tensorflow_version 2.x

### **模型相關參數**
到以下網址查詢模型與函數資訊
https://huggingface.co/transformers

In [0]:
PRETRAINED_MODEL_Folder ="model/"
PRETRAINED_Spiece_Name ="albert-base-spiece.model"
PRETRAINED_Spiece_Path =PRETRAINED_MODEL_Folder + PRETRAINED_Spiece_Name
trained_Model_Path = "trained_model"

## **引入需要⽤到的函式庫**

In [0]:
import os
import pickle
import torch
#from transformers import BertConfig, BertTokenizer, BertForSequenceClassification, AdamW
from transformers import AlbertConfig,AlbertTokenizer,AlbertForSequenceClassification,AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數

### **載入資料並將資料轉換成輸入格式(Token Embeddings、Segment Embeddings、Position Embeddings)**

In [0]:
def convert_data_to_feature(FileName):
    # 載入字典
    #tokenizer = BertTokenizer(vocab_file='bert-base-chinese-vocab.txt')
    #tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
    tokenizer = AlbertTokenizer.from_pretrained(PRETRAINED_Spiece_Path)

    # 載入資料
    Sentences = []
    SentencesId = []
    with open(FileName,'r',encoding='utf-8') as f:
        data = f.read()
    LS_pairs = data.split("\n")
    print('長度:',len(LS_pairs))
    #因為 label 與sentence在同⼀筆資料，所以我們將他們分開並個別存起來
    for LS_pair in LS_pairs:
        if LS_pair != "":
            try:
                #print(LS_pair)#2  1 A series of escapades demonstrating the adage that what is good for the goose 2
                LS_ary = LS_pair.split("\t")  #用tsv的【\t】來分割欄位
                SId=LS_ary[0] #句子id        
                S = LS_ary[2] #句子
                SentencesId.append(int(SId))  
                Sentences.append(S)
            except:
                continue
    
    assert len(Sentences)  == len(SentencesId)
    print('Sentences:',Sentences[0:20])
    # BERT input embedding
    max_seq_len = 0     # 紀錄最大長度
    input_ids = []
    original_length = []    # 紀錄原本長度
    #==轉換成BERT input 需要的格式===============================================
    for S in Sentences:
        # 將句子切割成一個個token
        word_piece_list = tokenizer.tokenize(S)
        # 將token轉成字典中的id
        input_id = tokenizer.convert_tokens_to_ids(word_piece_list)
        # 補上[CLS]和[SEP]
        input_id = tokenizer.build_inputs_with_special_tokens(input_id)

        if(len(input_id)>max_seq_len):
            max_seq_len = len(input_id)
        input_ids.append(input_id)
        #反向查詢(還原)
        #print(tokenizer.convert_ids_to_tokens(tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(input_id)))))
    #==轉換成BERT input 需要的格式  END==========================================
    
    print("最長句子長度:",max_seq_len)
    
    assert max_seq_len <= 512 # 小於BERT-base長度限制
    max_seq_len = 512

    #統⼀輸入資料的長度==========================================================
    # 補齊長度
    for c in input_ids:
        # 紀錄原本長度
        length = len(c)
        original_length.append(length)
        while len(c)<max_seq_len:
            c.append(0)
    #⽣成每⼀句的type_id
    token_type_ids = [[0]*max_seq_len for i in range(len(Sentences))]        # token_type_ids # 儲存的是句子的id，id為0就是第一句，id為1就是第二句
    attention_mask = []                                                      # attention_mask # 1代表是真實的單詞id，0代表補齊長度
    for i in range(len(Sentences)):
        attention_id = []
        for j in range(original_length[i]):
            attention_id.append(1)
        while len(attention_id)<max_seq_len:
            attention_id.append(0)
        attention_mask.append(attention_id)

    assert len(input_ids) == len(token_type_ids) and len(input_ids) == len(attention_mask) and len(input_ids)== len(SentencesId)

    #為了使⽤⽅便，把三個input embedding 和句子id 包成 object (預測無label) ======
    data_features = {'input_ids':input_ids,
                    'token_type_ids':token_type_ids,
                    'attention_mask':attention_mask,
                    'sentences_ids':SentencesId}

    print('sentences_ids:',SentencesId[0:10])
    print('input_ids:',input_ids[0:10])
    return data_features

### **輸入格式轉Dataset**

In [0]:
def makeDataset(data_feature):
    input_ids = data_feature['input_ids']
    token_type_ids = data_feature['token_type_ids']
    attention_mask = data_feature['attention_mask']
    sentences_ids = data_feature['sentences_ids']

    all_input_ids = torch.tensor([input_id for input_id in input_ids], dtype=torch.long)
    all_token_type_ids = torch.tensor([token_type_id for token_type_id in token_type_ids], dtype=torch.long)
    all_attention_mask_ids = torch.tensor([attention_id for attention_id in attention_mask], dtype=torch.long)
    all_sentences_ids = torch.tensor([sentences_id for sentences_id in sentences_ids], dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_token_type_ids, all_attention_mask_ids,all_sentences_ids)

    return dataset

## **確定 Embedding 是否正確**

In [43]:
#tokenizer = BertTokenizer(vocab_file='bert-base-chinese-vocab.txt')
# 取得此預訓練模型所使用的 tokenizer
#tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
tokenizer = AlbertTokenizer.from_pretrained(PRETRAINED_Spiece_Path)
data_features = convert_data_to_feature('predict_data.txt')

print(data_features['input_ids'][43])
print(tokenizer.convert_ids_to_tokens(data_features['input_ids'][43]))
print(data_features['token_type_ids'][43])
print(data_features['attention_mask'][43])
print(data_features['sentences_ids'][43])

Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated


長度: 66293
Sentences: ['An intermittently pleasing but mostly routine effort .', 'An intermittently pleasing but mostly routine effort', 'An', 'intermittently pleasing but mostly routine effort', 'intermittently pleasing but mostly routine', 'intermittently pleasing but', 'intermittently pleasing', 'intermittently', 'pleasing', 'but', 'mostly routine', 'mostly', 'routine', 'effort', '.', "Kidman is really the only thing that 's worth watching in Birthday Girl , a film by the stage-trained Jez Butterworth -LRB- Mojo -RRB- that serves as yet another example of the sad decline of British comedies in the post-Full Monty world .", 'Kidman', "is really the only thing that 's worth watching in Birthday Girl , a film by the stage-trained Jez Butterworth -LRB- Mojo -RRB- that serves as yet another example of the sad decline of British comedies in the post-Full Monty world .", "is really the only thing that 's worth watching in Birthday Girl , a film by the stage-trained Jez Butterworth -LRB- Moj

### **預測**

In [0]:
#predict 時把輸入的⽂字轉成輸入格式
def to_input_id(sentence_input):
    return tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence_input)))

In [45]:

import numpy as np

# setting device
device = torch.device('cuda')


#Step1:將訓練資料讀入並且組建BERT輸入格式 Start===================================
test_data_feature = convert_data_to_feature('predict_data.txt')
test_dataset = makeDataset(test_data_feature)
 #Step1:將訓練資料讀入並且組建BERT輸入格式 END====================================

#Step2:將組建好的輸入格式轉換成tensor格式，並且建立dataset與dataloader Srart======
test_dataloader = DataLoader(test_dataset ,batch_size=1 ,shuffle=True)
#Step2:將組建好的輸入格式轉換成tensor格式，並且建立dataset與dataloader END========

#將輸入的句⼦轉為wordpices格式並載入我們train好的模型
# 取得此預訓練模型所使用的 tokenizer
tokenizer = AlbertTokenizer.from_pretrained(PRETRAINED_Spiece_Path)

#config = AlbertConfig.from_pretrained(PRETRAINED_MODEL_Path,num_labels = 5)  #這步驟非常正要,要指定label數,不然預設是二分類
config = AlbertConfig.from_json_file(trained_Model_Path+'/config.json') #預測時用訓練產出的設定檔
model = AlbertForSequenceClassification.from_pretrained(trained_Model_Path+'/pytorch_model.bin', config = config) #預測時用訓練產出的模型

model.to(device)
model.eval()

f = open('submission.csv','w',encoding='utf-8')
f.write('PhraseId,Sentiment\n')
print('start predict')
a=0
for batch_dict in test_dataloader:
    batch_dict = tuple(t.to(device) for t in batch_dict)  #每⼀個Batch把資料從CPU RAM 推到 GPU RAM
    #print(batch_dict[0].shape)
    #順序對應產⽣Dataset那裡的dataset = TensorDataset(all_input_ids, all_token_type_ids,all_attention_mask_ids,all_sentences_ids)
    phrase_ids=batch_dict[3]
    outputs = model(
        input_ids = batch_dict[0]
        )
    logits = outputs[:2]
    
    for i in range(len(logits)):
        p_id = phrase_ids[i].item()
        a=a+1
        #if a>50:break
        predicts = logits[i]       
        max_val = torch.max(predicts)
        
        predict_label = (predicts == max_val).nonzero().cpu().numpy()[0][1] # 在第1維度取最大值並返回索引值
        #print(predict_label)
        f.write(str(p_id) + ',' + str(predict_label) + '\n')

print('end predict')
print(a)
f.close()


Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated


長度: 66293
Sentences: ['An intermittently pleasing but mostly routine effort .', 'An intermittently pleasing but mostly routine effort', 'An', 'intermittently pleasing but mostly routine effort', 'intermittently pleasing but mostly routine', 'intermittently pleasing but', 'intermittently pleasing', 'intermittently', 'pleasing', 'but', 'mostly routine', 'mostly', 'routine', 'effort', '.', "Kidman is really the only thing that 's worth watching in Birthday Girl , a film by the stage-trained Jez Butterworth -LRB- Mojo -RRB- that serves as yet another example of the sad decline of British comedies in the post-Full Monty world .", 'Kidman', "is really the only thing that 's worth watching in Birthday Girl , a film by the stage-trained Jez Butterworth -LRB- Mojo -RRB- that serves as yet another example of the sad decline of British comedies in the post-Full Monty world .", "is really the only thing that 's worth watching in Birthday Girl , a film by the stage-trained Jez Butterworth -LRB- Moj

Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated


start predict
end predict
66292
